# Reading csv file into panda dataframes and converting dataframe into json file

In [1]:
import pandas as pd
import json
df1 = pd.read_csv("E:/CS587/Food_Inspections.csv")

In [2]:
s = df1.to_json(orient='records')

In [3]:
cont = json.loads(s)

In [6]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch()

# Inserting the data into Elastic Search

In [10]:

actions = []
for data in cont:
    action = {
        "_index": "food_inspection_2",
        "_type": "food",
        "_id": data['Inspection ID'],
        "_source": {
            "any":data
            }
        }
    actions.append(action)

helpers.bulk(es, actions)

(164953, [])

# Qurey to fetch the data from elastic Search

In [98]:
doc = {
        'size' : 10000,
        'query': {
            'bool': {
                    'must' : [{'match' : {'any.Results': 'Fail'}},{'match' : {'any.Risk': 'Risk 1 (High)'}},
                            {"query_string": {
                                            "query": "*hild*",
                                            "fields": ["any.Facility Type","any.Violations","any.DBA Name"]
                                             }
                            }
           
                            ]              
                   }
                }
        }
res = es.search(index='food_inspection_2', body=doc,scroll='1h')

In [99]:
sid = res['_scroll_id']
scroll_size = res['hits']['total']

# Populating the data into local variable

In [100]:
count = 0
final_location_ll = []
while(scroll_size > 0):

    for doc in res['hits']['hits']:
        location_ll = []
        results = doc['_source']['any']
        count = count +1
        if 'Latitude' in results.keys():
            if 'Longitude' in results.keys():
                if 'Address' in results.keys():
                    if(results['Latitude'] != None and results['Longitude'] != None  and results['Address'] != None):

                        
                        location_ll.append(float(results['Latitude']))
                        location_ll.append(float(results['Longitude']))
                        final_location_ll.append(location_ll)
    res = es.scroll(scroll_id = sid, scroll = '2m')
    sid = res['_scroll_id']
    scroll_size = len(res['hits']['hits'])  
print(count)

803


In [101]:
len(final_location_ll)

803

# Importing folium to plot the map

In [95]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

print(folium.__version__)

0.5.0


In [96]:
m = folium.Map([41.90293279, -87.70769386], 
               zoom_start=11)
m

# For generating HeatMap

In [97]:
m.add_child(plugins.HeatMap(final_location_ll[:40000], radius=15))
m